In [1]:
from shapely.geometry import Point
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio
import dask.dataframe as dd
import fastparquet
from shapely.ops import nearest_points

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

#finish indicator
wow = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = wow, autoplay = True) 

In [4]:
taxi_zone = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp')#.to_crs('epsg:4326')
taxi_zone = taxi_zone.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

zip_code = gpd.read_file('../../Data/ZIP CODES SHAPE/ZIP_CODE_040114.shp')#.to_crs('epsg:4326')
gpd.sjoin(gpd.GeoDataFrame(crs='epsg:4326',
    geometry=[Point(-73.966, 40.78)]), 
    zip_code, how='left', op='within')

complete

In [5]:
rides = pd.read_csv('../../Data/yellow_tripdata_2015-01.csv')#.to_crs('epsg:4326')

complete

In [6]:
rides

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [8]:
taxi_points = taxi_zone.copy()
taxi_points['geometry'] = taxi_points['geometry'].centroid

In [9]:
taxi_points=taxi_points.to_crs('epsg:4326')

In [10]:
taxi_points

,zone,LocationID,borough,geometry
0,Newark Airport,1,EWR,POINT (-74.17400 40.69183)
1,Jamaica Bay,2,Queens,POINT (-73.83130 40.61675)
2,Allerton/Pelham Gardens,3,Bronx,POINT (-73.84742 40.86447)
3,Alphabet City,4,Manhattan,POINT (-73.97697 40.72375)
4,Arden Heights,5,Staten Island,POINT (-74.18849 40.55266)
...,...,...,...,...
258,Woodlawn/Wakefield,259,Bronx,POINT (-73.85222 40.89793)
259,Woodside,260,Queens,POINT (-73.90631 40.74423)
260,World Trade Center,261,Manhattan,POINT (-74.01302 40.70914)
261,Yorkville East,262,Manhattan,POINT (-73.94651 40.77593)


In [11]:
taxi_points['lat_lng']=taxi_points['geometry']

zip_points['lat_lng']=zip_points['geometry']

In [12]:
taxi_points['lat_lng']=taxi_points['lat_lng'].astype(str)

zip_points['lat_lng']=zip_points['lat_lng'].astype(str)

In [13]:
taxi_points_split=taxi_points['lat_lng'].str.strip('POINT(') \
                      .str.strip('(') \
                      .str.strip(')') \
                      .str.split(' ', expand=True)                   \
                     .rename(columns={1:'lat', 2:'lon'}) 

zip_points_split=zip_points['lat_lng'].str.strip('POINT(') \
                      .str.strip('(') \
                      .str.strip(')') \
                      .str.split(' ', expand=True)                   \
                     .rename(columns={1:'lat', 2:'lon'}) 

In [15]:
taxi_points=taxi_points.join(taxi_points_split)

zip_points=zip_points.join(taxi_points_split)

In [16]:
taxi_points['lat']=taxi_points['lat'].str.strip('(')

zip_points['lat']=zip_points['lat'].str.strip('(')

In [17]:
taxi_points=taxi_points[['zone','LocationID','borough','geometry','lat','lon']]

zip_points=zip_points[['ZIPCODE','PO_NAME','COUNTY','geometry','lat','lon']]

In [18]:
taxi_points

,zone,LocationID,borough,geometry,lat,lon
0,Newark Airport,1,EWR,POINT (-74.17400 40.69183),-74.17400137278653,40.69183011107549
1,Jamaica Bay,2,Queens,POINT (-73.83130 40.61675),-73.83130005002184,40.61674557260063
2,Allerton/Pelham Gardens,3,Bronx,POINT (-73.84742 40.86447),-73.84742177366226,40.86447318881057
3,Alphabet City,4,Manhattan,POINT (-73.97697 40.72375),-73.97696810617441,40.72375155202172
4,Arden Heights,5,Staten Island,POINT (-74.18849 40.55266),-74.18848505675093,40.55265861459782
...,...,...,...,...,...,...
258,Woodlawn/Wakefield,259,Bronx,POINT (-73.85222 40.89793),-73.85221508361175,40.89793162840499
259,Woodside,260,Queens,POINT (-73.90631 40.74423),-73.90630713230253,40.74423296307197
260,World Trade Center,261,Manhattan,POINT (-74.01302 40.70914),-74.01302265094951,40.70913843565989
261,Yorkville East,262,Manhattan,POINT (-73.94651 40.77593),-73.94651031378342,40.77593173933266


In [19]:
rides

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


taxi_points=taxi_points.rename(columns = {'lat1':'lat','lon1':'lon'})

In [20]:
rides=rides.rename(columns = {'pickup_latitude':'lat','pickup_longitude':'lon'})

In [22]:
rides

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,lon,lat,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [23]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12748986 entries, 0 to 12748985
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   lon                    float64
 6   lat                    float64
 7   RateCodeID             int64  
 8   store_and_fwd_flag     object 
 9   dropoff_longitude      float64
 10  dropoff_latitude       float64
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  float64
 18  total_amount           float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.8+ GB


In [24]:
taxi_points[['lat','lon']]=taxi_points[['lat','lon']].astype(float)

zip_points[['lat','lon']]=zip_points[['lat','lon']].astype(float)

In [25]:
def dist(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [32]:
def find_nearest(lat, lon):
    distances = taxi_points.apply(
        lambda row: dist(lat, lon, row['lat'], row['lon']), 
        axis=1)
    return taxi_points.loc[distances.idxmin(), 'LocationID']

In [33]:
rides['LocationID'] = rides.apply(
    lambda row: find_nearest(row['lat'], row['lon']), 
    axis=1)

taxi_points.head()

KeyboardInterrupt: 

In [ ]:
taxi_points['ZIPCODE'].nunique()

In [ ]:
taxi_points['LocationID'].nunique()

In [ ]:
taxi_points.crs

In [ ]:
taxi_zone_for_join = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')

In [ ]:
taxi_points_for_join = taxi_points[['LocationID','ZIPCODE']]

In [ ]:
merge=taxi_zone_for_join.merge(taxi_points_for_join)

In [ ]:
merge.to_file('../../Data/Final_Shape_test3.shp')

merge.info()

merge['ZIPCODE']=merge['ZIPCODE'].astype(int)

taxi_points['closest'] = taxi_points.apply(
    lambda row: find_nearest(row['lat'], row['lon']), 
    axis=1)
# To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
taxi_points.head()

trips= dd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-01.csv',dtype={' tolls_amount': 'float64'})

trips['pickup_taxizone_id'] = trips.map_partitions(
    assign_taxi_zones, " pickup_longitude", " pickup_latitude",
    "pickup_taxizone_id", meta=('pickup_taxizone_id', np.float64))
trips['dropoff_taxizone_id'] = trips.map_partitions(
    assign_taxi_zones, " dropoff_longitude", " dropoff_latitude",
    "dropoff_taxizone_id", meta=('dropoff_taxizone_id', np.float64))
trips[['pickup_taxizone_id', 'dropoff_taxizone_id']].head()

#assume_missing=True

start_time = time.time()

df.to_parquet('df_2014-01.parquet', has_nulls=True,
     object_encoding='json', compression="SNAPPY")
df = dd.read_parquet('df_2014-015.parquet',
    columns=['pickup_taxizone_id', 'dropoff_taxizone_id'])

print("--- %s seconds ---" % (time.time() - start_time))

complete

In [ ]:
print("--- %s seconds ---" % (time.time() - full_time))

In [36]:
rides

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,lon,lat,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12748981,1,2015-01-10 19:01:44,2015-01-10 19:05:40,2,1.00,-73.951988,40.786217,1,N,-73.953735,40.775162,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
12748982,1,2015-01-10 19:01:44,2015-01-10 19:07:26,2,0.80,-73.982742,40.728184,1,N,-73.974976,40.720013,1,6.0,0.0,0.5,2.00,0.0,0.3,8.80
12748983,1,2015-01-10 19:01:44,2015-01-10 19:15:01,1,3.40,-73.979324,40.749550,1,N,-73.969101,40.787800,2,13.5,0.0,0.5,0.00,0.0,0.3,14.30
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.30,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55
